In [1]:
#Update Tensorflow
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

df = pd.read_excel('/home/jh/ww/data/Pi Master Final 251017.xlsx')

#df_final is the desired data wth file name column, and the rest are one hot for labels
category = df['Category ']
category = df['Category '].str.lower()
category = category.str.strip(to_strip=None)
category.replace('professions & industries', 'industries & professions',inplace=True)
category = pd.get_dummies(category)
newdf = df[['File Name ']].copy()

df_final = newdf.join(category)

x_y_tup = {}

x = []
y = []

#Separating the df_final into image name:label, as a dictionary
for df in df_final.as_matrix():
    filename, labels = df[0], df[1:]
    assert filename not in x_y_tup
    x_y_tup[filename] = labels


#folder = "S3 BUCKET"
folder = '/home/jh/ww/imgs'

for image_title in os.listdir(folder):
    if image_title in x_y_tup:
        image = Image.open(os.path.join(folder, image_title))
        resized_image = Image.Image.resize(image, [400, 400])
        #x now contains the resized image as an array
        x.append(np.array(resized_image))
        #y now contains the correspinding labels
        y.append(x_y_tup[image_title])

    else:
        # each_image is a new image that we didnt expect
        assert False
           
#X = np.reshape(x, (-1, 400, 400,3)) / 255.
y = np.array(y)
#X = X.astype('float32')
print(y)

/home/jh/anaconda3/envs/pischool/lib/python3.6/site-packages/PIL/Image.py:2244: DecompressionBombWarning: Image size (164045592 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


[[0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 1 0 0]]


In [8]:
#Setting up layers
def layers(X):
    
    first_layer = tf.reshape(X, shape = [-1, 400, 400, 3])
    conv1 = tf.layers.conv2d(first_layer, filters = 30, kernel_size = [3, 3], padding = 'SAME', activation = tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = [2, 2], strides = 2)
    flattened = tf.reshape(pool1,[-1, (200*200*30)])
    fc1 = tf.layers.dense(inputs = flattened, units = 1024, activation = tf.nn.relu)
    dropout = tf.layers.dropout(inputs = fc1, rate = 0.4)
    logits = tf.layers.dense(inputs = dropout, units = 6)
    
    return logits

In [14]:
#defining placeholders
x = tf.placeholder(tf.float32, (None, 400, 400, 3))
y = tf.placeholder(tf.float32, None)
rate = 0.001
logits = layers(x)

In [16]:
#Loss and backprop
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=rate)
training_operation = optimizer.minimize(loss_operation)

In [18]:
# 1 is to say just choose the top 1
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
#To save the model
saver = tf.train.Saver()

In [22]:
#Evaluating
#Calling accuracy_operation, which called the correct_prediction function

BATCH_SIZE = 1

def evaluate(X_data, y_data):
    
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x:batch_x, y:batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

